Red neuronal convolutiva para procesamiento de imagenes en reconocimiento de digitos.

Con este modelo se logró un 99.428 % top 25%

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import random
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from tensorflow import keras
from keras import losses
from keras import metrics
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

# Plot Confusion matrix function
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],horizontalalignment="center",color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

import os
print(os.listdir("../input"))

In [ ]:
#Importa las imagenes 
train_df            = pd.read_csv("../input/mnist-digit-recognizer/train.csv")
unlabeled_images_df = pd.read_csv("../input/mnist-digit-recognizer/test.csv")

#Para el conjunto de entrenamiento separa la imagen de su etiqueta de clase
train_images_df     = train_df.iloc[:,1:]
train_labels_df     = train_df.iloc[:,0:1]

#Convierte los datos de pandas df a numpy array
labeled_images      = train_images_df.values
labels              = train_labels_df.values
unlabeled_images    = unlabeled_images_df.values

#Convierte la etiqueta de clase en multicategorical
one_hot_labels      = keras.utils.to_categorical(labels, num_classes=10)

#Normaliza las imagenes de rango (0, 255) al rango (0,1) 
labeled_images      = labeled_images/255
unlabeled_images    = unlabeled_images/255

#Cambia la forma de las imagenes de vector size = 784 a matrix size = (28,28,1)
labeled_images      = np.reshape(labeled_images,(labeled_images.shape[0],28,28,1))
unlabeled_images    = np.reshape(unlabeled_images,(unlabeled_images.shape[0],28,28,1))

In [ ]:
#Imagen de ejemplo
imagen = labeled_images[250,:,:,0]
plt.imshow(imagen)

In [ ]:
#modelo
model = keras.Sequential()

model.add(keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation = "relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation = "softmax"))

In [ ]:
random_seed = 2

#Define el tamaño del conjunto con el que se va a entrenar la red
test_set_percentage  = 0.25

#Define si se trabaja con output size = (1,10) o output size = (1)
sparse = False

if(sparse == True):
    train_images,test_images,train_labels,test_labels = train_test_split(labeled_images, labels, test_size = test_set_percentage, random_state=random_seed)
else:
    train_images,test_images,train_labels,test_labels = train_test_split(labeled_images, one_hot_labels, test_size = test_set_percentage, random_state=random_seed)
    
if(sparse == True):
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
else:
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics=['accuracy'])

In [ ]:
data_gen = keras.preprocessing.image.ImageDataGenerator(featurewise_center=False, 
                                                        samplewise_center=False, 
                                                        featurewise_std_normalization=False, 
                                                        samplewise_std_normalization=False, 
                                                        zca_whitening=False, 
                                                        zca_epsilon=1e-06, 
                                                        rotation_range=30, 
                                                        width_shift_range=0.1, 
                                                        height_shift_range=0.1, 
                                                        brightness_range=None, 
                                                        shear_range=0.1, 
                                                        zoom_range=0.1, 
                                                        channel_shift_range=0.0, 
                                                        fill_mode='nearest', 
                                                        cval=0.0, 
                                                        horizontal_flip=False, 
                                                        vertical_flip=False, 
                                                        rescale=0, 
                                                        preprocessing_function=None, 
                                                        data_format=None, 
                                                        validation_split=0.0, 
                                                        dtype=None)

In [ ]:
data_gen.fit(train_images)

In [ ]:
#Entrena el modelo con 10 epochs, pero originalmente se corrio con 40 epochs
fit_history = model.fit(data_gen.flow(train_images, train_labels, batch_size=32), 
                        epochs=10, 
                        validation_data=(test_images,test_labels),
                        verbose=2)

In [ ]:
acc      = fit_history.history['accuracy']
val_acc  = fit_history.history['val_accuracy']
loss     = fit_history.history['loss']
val_loss = fit_history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# summarize model.
model.summary()

In [ ]:
data_gen = keras.preprocessing.image.ImageDataGenerator(featurewise_center=False, 
                                                        samplewise_center=False, 
                                                        featurewise_std_normalization=False, 
                                                        samplewise_std_normalization=False, 
                                                        zca_whitening=False, 
                                                        zca_epsilon=1e-06, 
                                                        rotation_range=25, 
                                                        width_shift_range=0.1, 
                                                        height_shift_range=0.1, 
                                                        brightness_range=None, 
                                                        shear_range=0.1, 
                                                        zoom_range=0.1, 
                                                        channel_shift_range=0.0, 
                                                        fill_mode='nearest', 
                                                        cval=0.0, 
                                                        horizontal_flip=False, 
                                                        vertical_flip=False, 
                                                        rescale=0, 
                                                        preprocessing_function=None, 
                                                        data_format=None, 
                                                        validation_split=0.0, 
                                                        dtype=None)

In [ ]:
data_gen.fit(labeled_images)

In [ ]:
model.fit_generator(data_gen.flow(labeled_images, one_hot_labels, batch_size=36), epochs=10)

In [ ]:
model.evaluate(labeled_images, one_hot_labels, batch_size=36)

In [ ]:
# predict results
results = model.predict(unlabeled_images)

In [ ]:
print(results)

In [ ]:
# select the index with the maximum probability
results = np.argmax(results,axis=1)

results_serie = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results_serie],axis = 1)

In [ ]:
#Unlabeled data model predictions
submission